

# [Fine-Tuning Local Models with LoRA in Python](https://https://www.youtube.com/watch?v=XDOSVh9jJiA)


In [ ]:
!pip install -U bitsandbytes datasets accelerate loralib


In [ ]:
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
# Run the installation command again to ensure bitsandbytes is installed
# and updated to the latest version.
#!pip install -U bitsandbytes

# It is good practice to restart the kernel after installing new packages,
# especially if there are environment-related issues.
# If you are in a Jupyter environment, you can restart the kernel from the menu.
# If you are running this script directly, you might need to re-run the script.

# After restarting the kernel (if necessary), rerun the imports and the model loading code.
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, TaskType

model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
!pip install --upgrade fsspec
!pip install datasets==2.12.0


In [ ]:

data = load_dataset('openai/gsm8k', 'main', split='train[:200]')

In [ ]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [ ]:

tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

In [ ]:
training_args = TrainingArguments(
    output_dir = './tinyllama-lora-tuned',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    logging_steps = 20,
    save_strategy = 'epoch',
    report_to = 'none',
    remove_unused_columns = False,
    label_names = ["labels"]
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./tinyllama-lora-tuned-adapter-math")
tokenizer.save_pretrained("./tinyllama-lora-tuned-adapter-math")

# Evaluation

In [ ]:
from peft import PeftModel
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'
adapter_path = './tinyllama-lora-tuned-adapter-math'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
).eval()

tmp_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tuned_model = PeftModel.from_pretrained(tmp_model, adapter_path)
tuned_model = tuned_model.merge_and_unload().eval()

In [ ]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [ ]:
eval_ds = load_dataset('openai/gsm8k', 'main', split='train[:20]')
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['question', 'answer'])
eval_ds = eval_ds.with_format('torch')

In [ ]:
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, default_data_collator
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [ ]:

@torch.no_grad()
def compute_perplexity(model):
    import math

    losses = []

    for batch in eval_loader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        loss = model(**batch).loss
        losses.append(loss.item())

    return math.exp(sum(losses) / len(losses))

In [ ]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

In [ ]:
import random

raw_data = load_dataset('gsm8k', 'main', split='train[:20]')
refs = raw_data['answer']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
raw_data['question'][1]

In [ ]:
print(generate(base_model, raw_data['question'][1]))

In [ ]:
print(generate(tuned_model, raw_data['question'][1]))

In [ ]:
print(refs[1])

## Unseen Data

In [ ]:
eval_ds = load_dataset('openai/gsm8k', 'main', split='train[200:300]')
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['question', 'answer'])
eval_ds = eval_ds.with_format('torch')

In [ ]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [ ]:

print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

In [ ]:
raw_data = load_dataset('gsm8k', 'main', split='train[200:300]')
refs = raw_data['answer']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
raw_data['question'][0]

In [ ]:
print(generate(base_model, raw_data['question'][0]))

In [ ]:
print(generate(tuned_model, raw_data['question'][0]))

In [ ]:
print(refs[0])